# Aggregating Comsumption Data By Pressure Zone

In [1]:
# Import libraries
import numpy as np      
import pandas as pd    
from IPython.display import display
import matplotlib.pyplot as plt
import random
from scipy.stats.mstats import winsorize
import os

In [2]:
# Read in the pressure zone data 
filename = '../CleanedData/MeterNumberAndPressureZoneData.csv'
pressure_zone_data = pd.read_csv(filename)      
print(f"{filename} : file read into a pandas dataframe.")

../CleanedData/MeterNumberAndPressureZoneData.csv : file read into a pandas dataframe.


In [3]:
# Display the data
display(pressure_zone_data)
pressure_zone_data.info()

,MeterNumber,Pressure Zone
63,770020,0.0
65,770039,0.0
86,630061,0.0
93,770026,0.0
96,64280,0.0
...,...,...
34,54729627.0,12.0
35,54724768.0,12.0
36,54725515.0,12.0
37,54725929.0,12.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 22667 entries, 63 to 38
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MeterNumber    22667 non-null  object 
 1   Pressure Zone  22667 non-null  float64
dtypes: float64(1), object(1)
memory usage: 531.3+ KB


In [4]:
# Remove letters from the Meter Numbers, is this an issue? 
pressure_zone_data["MeterNumber"] = pressure_zone_data["MeterNumber"].apply(lambda x: x.replace('B', ''))
pressure_zone_data["MeterNumber"] = pressure_zone_data["MeterNumber"].apply(lambda x: x.replace('P', ''))
pressure_zone_data["MeterNumber"] = pressure_zone_data["MeterNumber"].apply(lambda x: x.replace('A', ''))
pressure_zone_data["MeterNumber"] = pressure_zone_data["MeterNumber"].apply(lambda x: x.replace('S', ''))

In [5]:
# Turn the data in the Meter Number column from strings to ints
pressure_zone_data["MeterNumber"] = pd.to_numeric(pressure_zone_data["MeterNumber"])
display(pressure_zone_data)
pressure_zone_data.info()

,MeterNumber,Pressure Zone
63,770020.0,0.0
65,770039.0,0.0
86,630061.0,0.0
93,770026.0,0.0
96,64280.0,0.0
...,...,...
34,54729627.0,12.0
35,54724768.0,12.0
36,54725515.0,12.0
37,54725929.0,12.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 22667 entries, 63 to 38
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MeterNumber    22667 non-null  float64
 1   Pressure Zone  22667 non-null  float64
dtypes: float64(2)
memory usage: 531.3 KB


### Access all of the daily consumption files to be able to iterate through all of them to work with the data in each

In [29]:
# Defining Helper Functions

def clean_aggregate_data(filename, folder):
    """
    This function extracts the date from the file name and then adds a date column to the datafile  
    """

    # Access the file data 
    directory = f'../EVWDDataRaw/Daily Consumption Data/{folder}/'
    file_data = pd.read_excel(directory + filename, skiprows=2)
    
    # Get the date from the file name 
    date = filename[:-5]
    
    # Drop unecessary columns and rows with na values 
    file_data = file_data.drop(["Account Name", "Address", "UOM", "Conv", "Meter Size", "MIU ID"], axis = 1)
    file_data = file_data.dropna()

    ### Turn Meter Number Column vals into ints ###
    dataTypeObj = file_data.dtypes['Meter Number']

    if str(dataTypeObj) != "float64":
        file_data["MeterNumber"] = file_data["Meter Number"].apply(lambda x: x.replace('a', ''))
        file_data["MeterNumber"] = pd.to_numeric(file_data["MeterNumber"])
    else:
        file_data["MeterNumber"] = pd.to_numeric(file_data["Meter Number"])

    ### Turn Account Numbers into Ints ###
    is_num = file_data["Account Number"] != "Account Number"
    file_data2 = file_data[is_num]
    file_data = file_data2

    file_data["AccountNumber"] = file_data["Account Number"].apply(lambda x: x.replace('-', ''))
    file_data = file_data.drop(["Account Number"], axis=1)
    file_data["Account Number"] = file_data["AccountNumber"].apply(lambda x: int(x))
    file_data = file_data.drop(["AccountNumber"], axis=1)
   
    ### Turn Consumption Column vals into ints ###
    file_data["consumption"] = pd.to_numeric(file_data["Consumption"])

    ### Remove negative values ###
    file_data = file_data[(file_data.consumption > 0)]


    ### Move the consumption column to the end of the data frame ###
    df1 = file_data.pop('consumption') 
    file_data['consumption']= df1 

    file_data = file_data.drop(["Meter Number", "Consumption"], axis = 1)

    ### Drop Accounts with outlier water usage ###
    accounts = ["0036009800", "0123114105", "0042027704", "0143019100", "0026017102", "0111027200", "0123069308", "0032146200", "0044017104", "0043019404", "0132024801", "0115020500", "0115020100", "0021021000" ]
    file_data = file_data.loc[~file_data["Account Number"].isin(accounts)]

    ### Merge the pressure zone data with the consumption data ###
    merged_data = pd.merge(left=file_data, right=pressure_zone_data, left_on='MeterNumber', right_on='MeterNumber')

    # Sum up consumption by Pressurezone
    zone_0_sum =  merged_data.loc[merged_data['Pressure Zone'] == 0, 'consumption'].sum()
    zone_1_sum =  merged_data.loc[merged_data['Pressure Zone'] == 1, 'consumption'].sum()
    zone_2_sum =  merged_data.loc[merged_data['Pressure Zone'] == 2, 'consumption'].sum()
    zone_3_sum =  merged_data.loc[merged_data['Pressure Zone'] == 3, 'consumption'].sum()
    zone_4_sum =  merged_data.loc[merged_data['Pressure Zone'] == 4, 'consumption'].sum()
    zone_5_sum =  merged_data.loc[merged_data['Pressure Zone'] == 5, 'consumption'].sum()
    zone_6_sum =  merged_data.loc[merged_data['Pressure Zone'] == 6, 'consumption'].sum()
    zone_7_sum =  merged_data.loc[merged_data['Pressure Zone'] == 7, 'consumption'].sum()
    zone_8_sum =  merged_data.loc[merged_data['Pressure Zone'] == 8, 'consumption'].sum()
    zone_9_sum =  merged_data.loc[merged_data['Pressure Zone'] == 9, 'consumption'].sum()
    zone_10_sum =  merged_data.loc[merged_data['Pressure Zone'] == 10, 'consumption'].sum()
    zone_11_sum =  merged_data.loc[merged_data['Pressure Zone'] == 11, 'consumption'].sum()
    zone_12_sum =  merged_data.loc[merged_data['Pressure Zone'] == 12, 'consumption'].sum()

    # Zone sums
    zone_sums = [zone_0_sum, zone_1_sum, zone_2_sum, zone_3_sum, zone_4_sum, zone_5_sum, zone_6_sum, zone_7_sum, zone_8_sum, zone_9_sum, zone_10_sum, zone_11_sum, zone_12_sum]

    # Pressure Zones 
    pressure_zones = [0,1,2,3,4,5,6,7,8,9,10,11,12]

    # and merge them by using zip().
    zone_and_sum = list(zip(pressure_zones, zone_sums))

    aggregated_by_zone = pd.DataFrame(zone_and_sum,
                  columns = ['PressureZone', 'Consumption'])

    ### Add a date column with the correct date ###
    aggregated_by_zone["Date"] = date

    ### Turn the Date into a numeric value for the Day of the Week ###
    aggregated_by_zone["Date"] = aggregated_by_zone["Date"] = aggregated_by_zone["date"] = pd.to_datetime(aggregated_by_zone["Date"])
    aggregated_by_zone["DayOfWeek"] = aggregated_by_zone["date"].dt.dayofweek
    aggregated_by_zone = aggregated_by_zone.drop(["date"], axis = 1)    

    return aggregated_by_zone


    


In [23]:
def stack_consumption_data(consumption_files, folder):
    """
    This function assigns a an int to each pressure zone and stacks all the pressure zone data
    inputs: the list of pressure zone file names, an empty data frame with the correct column names
    output: a data frame with to colums (Meter Number and Pressure Zone) that has all of the stacked pressure zone data
    """

    d = {"PressureZone": [], "Consumption": [], "Date": [], "DayOfWeek": []}
    df = pd.DataFrame(data = d)

    num_of_files = len(consumption_files)

    for num, filename in enumerate(consumption_files):
        aggregated_data = clean_aggregate_data(filename, folder)
        
        # Stack the current zone data frame under the previous zone's data 
        stacked_aggregated_consumption = pd.concat([df,aggregated_data])
        df = stacked_aggregated_consumption
        print("stacked")
        if num == num_of_files-1:
            return stacked_aggregated_consumption

In [32]:
# Get all file paths
consumption_data_files = os.listdir('../EVWDDataRaw/Daily Consumption Data/2021Data/March')
print(consumption_data_files)


['3-29-2021.xlsx', '3-3-2021.xlsx', '3-24-2021.xlsx', '3-30-2021.xlsx', '3-11-2021.xlsx', '3-5-2021.xlsx', '3-17-2021.xlsx', '3-22-2021.xlsx', '3-8-2021.xlsx', '3-2-2021.xlsx', '3-28-2021.xlsx', '3-31-2021.xlsx', '3-25-2021.xlsx', '3-10-2021.xlsx', '3-4-2021.xlsx', '3-16-2021.xlsx', '3-9-2021.xlsx', '3-23-2021.xlsx', '3-1-2021.xlsx', '3-13-2021.xlsx', '3-26-2021.xlsx', '3-7-2021.xlsx', '3-18-2021.xlsx', '3-20-2021.xlsx', '3-15-2021.xlsx', '3-12-2021.xlsx', '3-27-2021.xlsx', '3-6-2021.xlsx', '3-19-2021.xlsx', '3-21-2021.xlsx', '3-14-2021.xlsx']


In [34]:
# Stack the data
stacked_consumption_data = stack_consumption_data(consumption_data_files, "2021Data/March")
display(stacked_consumption_data)

stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked
stacked


,PressureZone,Consumption,Date,DayOfWeek
0,0.0,9500526.78,2021-03-29,0.0
1,1.0,2810.38,2021-03-29,0.0
2,2.0,279588.35,2021-03-29,0.0
3,3.0,1629.94,2021-03-29,0.0
4,4.0,3878.15,2021-03-29,0.0
...,...,...,...,...
8,8.0,4012145.21,2021-03-14,6.0
9,9.0,106083.17,2021-03-14,6.0
10,10.0,668.58,2021-03-14,6.0
11,11.0,1045647.50,2021-03-14,6.0


In [35]:
display(stacked_consumption_data)

,PressureZone,Consumption,Date,DayOfWeek
0,0.0,9500526.78,2021-03-29,0.0
1,1.0,2810.38,2021-03-29,0.0
2,2.0,279588.35,2021-03-29,0.0
3,3.0,1629.94,2021-03-29,0.0
4,4.0,3878.15,2021-03-29,0.0
...,...,...,...,...
8,8.0,4012145.21,2021-03-14,6.0
9,9.0,106083.17,2021-03-14,6.0
10,10.0,668.58,2021-03-14,6.0
11,11.0,1045647.50,2021-03-14,6.0


In [36]:
# Write current data frame into a new cvs file 
stacked_consumption_data.to_csv("March2021ZoneData.csv", index_label=False)